In [1]:
# Equivalent Python implementation of the given Java Pass-I Assembler

IS = {"ADD": "01", "SUB": "02", "MULT": "03", "MOVER": "04", "MOVEM": "05", "COMP": "06", "BC": "07", "DIV": "08", "READ": "09", "PRINT": "10"}
AD = {"START": "01", "END": "02"}
DL = {"DS": "01", "DC": "02"}
REG = {"AREG": "1", "BREG": "2", "CREG": "3", "DREG": "4"}

symtab = []
symtabAddr = {}
littab = []
littabAddr = {}
intermediate = []

def pass1(program):
    global symtab, symtabAddr, littab, littabAddr, intermediate
    lc = 0
    value = None

    for line in program:
        parts = line.replace(',', ' ').split()
        if not parts:
            continue

        if parts[0] in AD:
            # START / END
            if parts[0] == "START":
                lc = int(parts[1])
                intermediate.append(f"AD {AD['START']} C {lc}")
            elif parts[0] == "END":
                intermediate.append(f"AD {AD['END']}")
                # Assign addresses to literals
                for lit in littab:
                    littabAddr[lit] = lc
                    value = lit[2:-1]  # remove =''
                    intermediate.append(f"DL {DL['DC']} C {value}")
                    lc += 1

        elif parts[0] in IS:
            # Instruction without label
            ic = f"IS {IS[parts[0]]}"
            if len(parts) > 1 and parts[1] in REG:
                ic += f" RG {REG[parts[1]]}"

            if len(parts) > 2:
                if parts[2].startswith('='):
                    if parts[2] not in littab:
                        littab.append(parts[2])
                    idx = littab.index(parts[2])
                    ic += f" L {idx}"
                else:
                    if parts[2] not in symtab:
                        symtab.append(parts[2])
                    idx = symtab.index(parts[2])
                    ic += f" S {idx}"
            intermediate.append(ic)
            lc += 1

        else:
            # Label + DL (DS/DC)
            label = parts[0]
            if label not in symtab:
                symtab.append(label)
            symtabAddr[label] = lc

            if parts[1] in DL:
                if parts[1] == "DS":
                    intermediate.append(f"DL {DL['DS']} C {parts[2]}")
                    lc += int(parts[2])
                elif parts[1] == "DC":
                    intermediate.append(f"DL {DL['DC']} C {parts[2]}")
                    lc += 1


def main():
    program = [
        "START 200",
        "MOVER AREG, ONE",
        "ADD BREG, TWO",
        "ADD BREG, ='5'",
        "MOVEM CREG, RESULT",
        "ADD BREG, ='3'",
        "ONE DS 4",
        "TWO DC 2",
        "RESULT DS 1",
        "END"
    ]

    pass1(program)

    print("\nSYMTAB:")
    for i, sym in enumerate(symtab):
        print(f"{i}  {sym}  {symtabAddr.get(sym)}")

    print("\nLITTAB:")
    for i, lit in enumerate(littab):
        print(f"{i}  {lit}  {littabAddr.get(lit)}")

    print("\nIntermediate Code:")
    for line in intermediate:
        print(line)


if __name__ == "__main__":
    main()



SYMTAB:
0  ONE  205
1  TWO  209
2  RESULT  210

LITTAB:
0  ='5'  211
1  ='3'  212

Intermediate Code:
AD 01 C 200
IS 04 RG 1 S 0
IS 01 RG 2 S 1
IS 01 RG 2 L 0
IS 05 RG 3 S 2
IS 01 RG 2 L 1
DL 01 C 4
DL 02 C 2
DL 01 C 1
AD 02
DL 02 C 5
DL 02 C 3
